In [25]:
import pandas as pd
import requests
from datetime import datetime, timedelta

# Define start and end dates
end_date = datetime.today().strftime("%Y-%m-%d")
start_date = (datetime.today() - timedelta(days=5000)).strftime("%Y-%m-%d")
# Data imported from the AlphaVantage api 
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=NFLX&apikey=PY54KY77EHCHF89J&datatype=csv&outputsize=full&start_date={start_date}&end_date={end_date}'
r = requests.get(url)

if r.status_code == 200:
    
    data = pd.read_csv(url)

    print(data.head())
else:
    print("Failed to retrieve data. Status code:", r.status_code)


    timestamp     open    high       low   close   volume
0  2024-04-05  624.920  637.91  622.7100  636.18  3327195
1  2024-04-04  633.210  638.00  616.5800  617.14  3008557
2  2024-04-03  612.745  630.41  611.5000  630.08  2913989
3  2024-04-02  611.000  615.03  605.5101  614.21  2018210
4  2024-04-01  608.000  615.11  605.5710  614.31  2063845


In [26]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["timestamp"],
                                        open=data["open"], 
                                        high=data["high"],
                                        low=data["low"], 
                                        close=data["close"])])
figure.update_layout(title = "Netflix Stock Price Analysis", 
                     xaxis_rangeslider_visible=False)
figure.show()

In [29]:
x = data[["open", "high", "low", "volume"]]
y = data["close"]
x = x.to_numpy()
y = y.to_numpy()
y = y.reshape(-1, 1)

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, 
                                                test_size=0.2, 
                                                random_state=42)

In [30]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 128)            66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117619 (459.45 KB)
Trainable params: 117619 (459.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(xtrain, ytrain, batch_size=1, epochs=30, validation_data=(xtest, ytest))
print(history.history)


Epoch 1/30

4404/4404 [==============================] - 15s 3ms/step - loss: 4787.8389 - val_loss: 158.1839
Epoch 2/30
4404/4404 [==============================] - 15s 3ms/step - loss: 528.7976 - val_loss: 120.1924
Epoch 3/30
4404/4404 [==============================] - 13s 3ms/step - loss: 433.3870 - val_loss: 144.7224
Epoch 4/30
4404/4404 [==============================] - 14s 3ms/step - loss: 377.9561 - val_loss: 242.8547
Epoch 5/30
4404/4404 [==============================] - 11s 3ms/step - loss: 470.3739 - val_loss: 524.3699
Epoch 6/30
4404/4404 [==============================] - 14s 3ms/step - loss: 375.0894 - val_loss: 77.5337
Epoch 7/30
4404/4404 [==============================] - 11s 3ms/step - loss: 513.4436 - val_loss: 216.5644
Epoch 8/30
4404/4404 [==============================] - 12s 3ms/step - loss: 224.8446 - val_loss: 65.4502
Epoch 9/30
4404/4404 [==============================] - 16s 4ms/step - loss: 312.5443 - val_loss: 238.1746
Epoch 10/30
4404/4404 [==============

In [33]:
import numpy as np
#features = [Open, High, Low, Adj Close, Volume]
features = np.array([[401.970001, 427.700012, 398.200012, 20047500]])
model.predict(features)

1/1 [==============================] - 1s 664ms/step


array([[410.85846]], dtype=float32)